# Aissignment : Semantic Analysis
### Siddharth Shakya

# Data Exploration

In [1]:
import pandas as pd
import os
from nltk.corpus import stopwords
from IPython.display import display

from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,classification_report
import pickle

# 1 Basic preprocessing 
##   Remove stop words, special characters, digits and add labels

#### Before doing this I have manually separated few paragraphs that were not separated by new line and randomly split paragraphs that were too large than usual just by visual inspection

In [3]:
stopList = list(stopwords.words('english')) # Stopwords
spclCharList  = ['#','^','!','(',')','`','~','%','/','\\' ,'{','}',']','[','.',',',':',';','"','-','°','@']

fruit_file = os.path.join(os.getcwd(),"TrainData\\apple-fruit.txt")
company_file = os.path.join(os.getcwd(),"TrainData\\apple-computers.txt")
trainDataText = open(os.path.join(os.getcwd(),"TrainData\\TrainData.txt"),'w')

with open(fruit_file,'r',encoding="utf-8") as input:
    paragraphs = input.read().split("\n\n")

datum =''
for para in paragraphs:
    for ch in spclCharList:
        para = para.replace(ch,'')
    para = ''.join([i for i in para if not i.isdigit()])
    para = para.lower().strip().split()
    para = list(filter(lambda x: x != '', para))
    para = [word for word in para if word not in stopList]
    para = ' '.join(str(w) for w in para)
    datum += para +'\t'+'fruit'+'\n'

with open(company_file,'r',encoding="utf-8") as input:
    paragraphs = input.read().split("\n\n")

for para in paragraphs:
    for ch in spclCharList:
        para = para.replace(ch,'')
    para = ''.join([i for i in para if not i.isdigit()])
    para = para.lower().strip().split()
    para = list(filter(lambda x: x != '', para))
    para = [word for word in para if word not in stopList]
    para = ' '.join(str(w) for w in para)
    datum += para +'\t'+'company'+'\n'

trainDataText.write(datum)
trainDataText.close()

In [4]:
paragraphs = []
label = []
trainDataText = open(os.path.join(os.getcwd(),"TrainData\\TrainData.txt"),'r')

In [5]:
# Add lables 1-0
df = pd.DataFrame(columns=['paragraph', 'label'])
for line in trainDataText.readlines():
    line = line.rstrip('\n').split('\t')
    paragraphs.append(line[0])
    l = 1 if line[1] == 'company' else 0
    label.append(1 if line[1] == 'company' else 0)
    df = df.append({'paragraph':line[0],'label' :l},ignore_index=True)
#df

## lemmatize

In [6]:
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

In [7]:
print(df['paragraph'][:6].apply(lambda x: " ".join([lmtzr.lemmatize(word) for word in x.split()])))


0    article fruit technology company see apple inc...
1    typical apple scientific classification kingdo...
2    apple pomaceous fruit apple tree specie malus ...
3    known cultivar apple resulting range desired c...
4    million tonne apple grown worldwide china prod...
5    botanical information blossom fruit leaf apple...
Name: paragraph, dtype: object


In [8]:
df2 = pd.DataFrame(columns=['lmtzPara', 'label'])
df2['lmtzPara'] = df['paragraph'].apply(lambda x: " ".join([lmtzr.lemmatize(word) for word in x.split()]))
df2['label'] = df['label']

## Summarize 
### Frequnet words

In [9]:
df['paragraph'].describe()

count                                                   211
unique                                                  211
top       macintosh initially sold well followup sales s...
freq                                                      1
Name: paragraph, dtype: object

In [10]:
df2['lmtzPara'].describe()

count                                                   211
unique                                                  211
top       success powerbook product brought increasing r...
freq                                                      1
Name: lmtzPara, dtype: object

In [11]:
df2.groupby('label').describe()
#df2

lmtzPara                                                               
         count unique                                                top freq
label                                                                        
0           81     81  apple scab apple scab cause leaf develop olive...    1
1          130    130  following success macintosh lc apple introduce...    1

In [12]:
df.groupby('label').describe()

paragraph                                                               
          count unique                                                top freq
label                                                                         
0            81     81  among serious disease problems fireblight bact...    1
1           130    130  october apple india executives unveiled plan e...    1

## Top K words with average Tf-IDF scores for both class and overall data

In [13]:
ngram = (1,2)
maxf = 800

##### vectorize

In [14]:
import numpy as np

vectorizer = TfidfVectorizer("english",ngram_range = ngram, max_features = maxf)

lmtz_vectorizer = TfidfVectorizer("english",ngram_range = ngram, max_features = maxf)

X = vectorizer.fit_transform(paragraphs)
lmtz_X = lmtz_vectorizer.fit_transform(df2['lmtzPara'])

##### split class wise

In [15]:
# split TfIdf-matrix class wise
DataMatrix = X.toarray()
lmtz_DataMatrix = lmtz_X.toarray()

#non lemmatized
X_fruit = DataMatrix[0:81,:] # first 81 are fruit in data
X_company = DataMatrix[81:,:]

#lemmatized
lmtz_X_fruit = lmtz_DataMatrix[0:81,:]
lmtz_X_company = lmtz_DataMatrix[81:,:]

###### find avg

In [16]:
#non lemmatized
X_fruit_Avg = X_fruit.sum(axis=0)/X_fruit.shape[0]
X_company_Avg = X_company.sum(axis=0)/X_company.shape[0]
Data_Avg = DataMatrix.sum(axis=0)/DataMatrix.shape[0]

#lemmatized
lmtz_X_fruit_Avg = lmtz_X_fruit.sum(axis=0)/X_fruit.shape[0]
lmtz_X_company_Avg = lmtz_X_company.sum(axis=0)/X_company.shape[0]
lmtz_Data_Avg = lmtz_DataMatrix.sum(axis=0)/lmtz_DataMatrix.shape[0]

##### find index for top k

In [17]:
k=-10

topK_IndFruit = np.argsort(X_fruit_Avg)[k:]
topK_IndCompany = np.argsort(X_company_Avg)[k:]
topK_IndData = np.argsort(Data_Avg)[k:]

lmtz_topK_IndFruit = np.argsort(lmtz_X_fruit_Avg)[k:]
lmtz_topK_IndCompany = np.argsort(lmtz_X_company_Avg)[k:]
lmtz_topK_IndData = np.argsort(lmtz_Data_Avg)[k:]
topK_IndData,lmtz_topK_IndData

(array([444, 294, 271, 396,  20, 150, 499, 380,  66,  43], dtype=int64),
 array([796, 271, 584, 735, 294,  23, 499, 376, 154,  43], dtype=int64))

##### put index for max first

###### fruit

In [18]:
#put max first
topK_IndFruit = topK_IndFruit[::-1]
lmtz_topK_IndFruit = lmtz_topK_IndFruit[::-1]

(topK_IndFruit, lmtz_topK_IndFruit)

(array([ 66,  43, 294, 181, 732, 456, 733,  20, 584, 469], dtype=int64),
 array([ 43, 294, 735, 184, 762, 455, 223,  23, 647, 682], dtype=int64))

##### company

In [19]:
topK_IndCompany = topK_IndCompany[::-1]
lmtz_topK_IndCompany = lmtz_topK_IndCompany[::-1]

(topK_IndFruit, lmtz_topK_IndFruit)

(array([ 66,  43, 294, 181, 732, 456, 733,  20, 584, 469], dtype=int64),
 array([ 43, 294, 735, 184, 762, 455, 223,  23, 647, 682], dtype=int64))

###### full data

In [20]:
topK_IndData = topK_IndData[::-1]
lmtz_topK_IndData = lmtz_topK_IndData[::-1]

(topK_IndData, lmtz_topK_IndData)

(array([ 43,  66, 380, 499, 150,  20, 396, 271, 294, 444], dtype=int64),
 array([ 43, 154, 376, 499,  23, 294, 735, 584, 271, 796], dtype=int64))

##### get feature names

In [21]:
featureNames = vectorizer.get_feature_names()
lmtz_featureNames = lmtz_vectorizer.get_feature_names()
#(featureNames[:10],lmtz_featureNames[:10])

##### index to feature name

In [22]:
indexToName ={i:featureNames[i] for i in range(len(featureNames))}
lmtz_indexToName ={i:lmtz_featureNames[i] for i in range(len(lmtz_featureNames))}

##### top k features names

In [23]:
top_K_AvgTfIdf_fruit = [indexToName[i] for i in topK_IndFruit]
top_K_AvgTfIdf_company = [indexToName[i] for i in topK_IndCompany]
top_K_AvgTfIdf_Data    = [indexToName[i] for i in topK_IndData]

lmtz_top_K_AvgTfIdf_fruit = [lmtz_indexToName[i] for i in lmtz_topK_IndFruit]
lmtz_top_K_AvgTfIdf_company = [lmtz_indexToName[i] for i in lmtz_topK_IndCompany]
lmtz_top_K_AvgTfIdf_Data    = [lmtz_indexToName[i] for i in lmtz_topK_IndData]

##### make data frame

### Compare lemmatized vs non lemmatized top K for same class

In [24]:
dfTopk_cmpny = pd.DataFrame(columns=['top_K_AvgTfIdf_company', 'lmtz_top_K_AvgIdf_company'])
dfTopk_cmpny['top_K_AvgTfIdf_company'] = top_K_AvgTfIdf_company
dfTopk_cmpny['lmtz_top_K_AvgIdf_company']=lmtz_top_K_AvgTfIdf_company
dfTopk_cmpny

,top_K_AvgTfIdf_company,lmtz_top_K_AvgIdf_company
0,apple,apple
1,iphone,company
2,company,iphone
3,jobs,product
4,new,job
5,macintosh,computer
6,first,macintosh
7,ipad,new
8,introduced,first
9,products,store


In [25]:
(top_K_AvgTfIdf_fruit, lmtz_top_K_AvgTfIdf_fruit)
dfTopk_frt = pd.DataFrame(columns=['top_K_AvgTfIdf_fruit', 'lmtz_top_K_AvgTfIdf_fruit'])

dfTopk_frt['top_K_AvgTfIdf_fruit']=top_K_AvgTfIdf_fruit
dfTopk_frt['lmtz_top_K_AvgTfIdf_fruit']=lmtz_top_K_AvgTfIdf_fruit
dfTopk_frt

,top_K_AvgTfIdf_fruit,lmtz_top_K_AvgTfIdf_fruit
0,apples,apple
1,apple,fruit
2,fruit,tree
3,cultivars,cultivar
4,tree,variety
5,malus,malus
6,trees,disease
7,also,also
8,production,seed
9,may,state


In [26]:
(top_K_AvgTfIdf_Data, lmtz_top_K_AvgTfIdf_Data)
dfTopk_data = pd.DataFrame(columns=['top_K_AvgTfIdf_Data', 'lmtz_top_K_AvgTfIdf_Data'])
dfTopk_data['top_K_AvgTfIdf_Data'] = top_K_AvgTfIdf_Data
dfTopk_data['lmtz_top_K_AvgTfIdf_Data']=lmtz_top_K_AvgTfIdf_Data
dfTopk_data

,top_K_AvgTfIdf_Data,lmtz_top_K_AvgTfIdf_Data
0,apple,apple
1,apples,company
2,iphone,iphone
3,new,new
4,company,also
5,also,fruit
6,jobs,tree
7,first,product
8,fruit,first
9,macintosh,year


### top lemmatized and non lemmatized seem pretty much similar for both classes and over all data

## Cmpare top words for classes and over all data

### non lemmatized

In [27]:
dfTopk_comparison = pd.DataFrame(columns=['top_K_AvgTfIdf_Data', 'top_K_AvgTfIdf_fruit','top_K_AvgTfIdf_company'])
dfTopk_comparison['top_K_AvgTfIdf_Data']=top_K_AvgTfIdf_Data
dfTopk_comparison['top_K_AvgTfIdf_fruit']=top_K_AvgTfIdf_fruit
dfTopk_comparison['top_K_AvgTfIdf_company']=top_K_AvgTfIdf_company
dfTopk_comparison

,top_K_AvgTfIdf_Data,top_K_AvgTfIdf_fruit,top_K_AvgTfIdf_company
0,apple,apples,apple
1,apples,apple,iphone
2,iphone,fruit,company
3,new,cultivars,jobs
4,company,tree,new
5,also,malus,macintosh
6,jobs,trees,first
7,first,also,ipad
8,fruit,production,introduced
9,macintosh,may,products


### lemmatized

In [28]:
lmtz_dfTopk_comparison = pd.DataFrame(columns=['lmtz_top_K_AvgTfIdf_Data', 'lmtz_top_K_AvgTfIdf_fruit','lmtz_top_K_AvgTfIdf_company'])
lmtz_dfTopk_comparison['lmtz_top_K_AvgTfIdf_Data']=lmtz_top_K_AvgTfIdf_Data
lmtz_dfTopk_comparison['lmtz_top_K_AvgTfIdf_fruit']=lmtz_top_K_AvgTfIdf_fruit
lmtz_dfTopk_comparison['lmtz_top_K_AvgTfIdf_company']=lmtz_top_K_AvgTfIdf_company
lmtz_dfTopk_comparison

,lmtz_top_K_AvgTfIdf_Data,lmtz_top_K_AvgTfIdf_fruit,lmtz_top_K_AvgTfIdf_company
0,apple,apple,apple
1,company,fruit,company
2,iphone,tree,iphone
3,new,cultivar,product
4,also,variety,job
5,fruit,malus,computer
6,tree,disease,macintosh
7,product,also,new
8,first,seed,first
9,year,state,store


## Top frequent words and ones with top Tf-IDF  are quite different across classes and overall data, so a linear model for classification may do well

# 2 Model Building

### 2.1 Fitting  Support Vector Machine (non lemmatized data)

In [48]:
ngram = (1,1)
maxf= 800

In [49]:
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

##### vectorizer for both lematize and non lematized

In [31]:
vectorizer = TfidfVectorizer("english",ngram_range = ngram, max_features = maxf)
lmtz_vectorizer = TfidfVectorizer("english",ngram_range = ngram, max_features = maxf)

X = vectorizer.fit_transform(paragraphs)

lmtz_X = lmtz_vectorizer.fit_transform(df2['lmtzPara'])

In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(X, label, stratify=label, test_size=.4)

In [33]:
lmtz_X_train, lmtz_X_test, lmtz_Y_train, lmtz_Y_test = train_test_split(lmtz_X, label, stratify=label, test_size=.4)

In [34]:
hyper_parameters = [{'kernel': ['linear'], 'gamma': [1e-3, 1e-4],'C': [1, 10, 100, 1000]}]

print('number of features : ', len(vectorizer.get_feature_names()))
#vectorizer.get_feature_names()

number of features :  800


### Train non lemmatized

In [35]:
classifier = GridSearchCV(SVC(), hyper_parameters, cv=5, scoring='f1')
classifier.fit(X_train, Y_train)
best_param = classifier.best_params_
best_classifier = classifier.best_estimator_
print('best param : ',best_param)

best param :  {'kernel': 'linear', 'gamma': 0.001, 'C': 1}


g:\anaconda3\envs\python35\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


### Train lemmatized

In [36]:
lmtz_classifier = GridSearchCV(SVC(), hyper_parameters, cv=5, scoring='f1')
lmtz_classifier.fit(X_train, Y_train)
lmtz_best_param = lmtz_classifier.best_params_
lmtz_best_classifier = lmtz_classifier.best_estimator_
print('best param : ',lmtz_best_param)

best param :  {'kernel': 'linear', 'gamma': 0.001, 'C': 1}


g:\anaconda3\envs\python35\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


### Predic non lemmatized

In [37]:
predicted = classifier.predict(X_test)
print (classifier.best_params_)
print ("Accuracy Score : ",accuracy_score(Y_test, predicted))
print ("F1 Score : ",f1_score(Y_test, predicted))
print ("Confusion Matrix : \n",confusion_matrix(Y_test, predicted))
#print ('\nClasification report:\n', classification_report(Y_test, predicted))

{'kernel': 'linear', 'gamma': 0.001, 'C': 1}
Accuracy Score :  0.9647058823529412
F1 Score :  0.9719626168224299
Confusion Matrix : 
 [[30  3]
 [ 0 52]]


### predict lemmatized

In [38]:
lmtz_predicted = lmtz_classifier.predict(lmtz_X_test)
print (classifier.best_params_)
print ("Accuracy Score : ",accuracy_score(lmtz_Y_test, lmtz_predicted))
print ("F1 Score : ",f1_score(lmtz_Y_test, lmtz_predicted))
print ("Confusion Matrix : \n",confusion_matrix(lmtz_Y_test, lmtz_predicted))
#print ('\nClasification report:\n', classification_report(lmtz_Y_test, lmtz_predicted))

{'kernel': 'linear', 'gamma': 0.001, 'C': 1}
Accuracy Score :  0.6941176470588235
F1 Score :  0.7868852459016394
Confusion Matrix : 
 [[11 22]
 [ 4 48]]


### Train lemmatized with rbf kernel as one of the hyper parameter

In [39]:
hyper_parameters = [{'kernel': ['linear'], 'gamma': [1e-3, 1e-4],'C': [1, 10, 100, 1000]},
                   {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],'C': [1, 10, 100, 1000]}]

In [40]:
lmtz_classifier = GridSearchCV(SVC(), hyper_parameters, cv=5, scoring='f1')
lmtz_classifier.fit(X_train, Y_train)
lmtz_best_param = lmtz_classifier.best_params_
lmtz_best_classifier = lmtz_classifier.best_estimator_
print('best param : ',lmtz_best_param)

best param :  {'kernel': 'linear', 'gamma': 0.001, 'C': 1}


g:\anaconda3\envs\python35\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


### Predict lemmatized with rbf kernel as hyper parameter

In [41]:
lmtz_predicted = lmtz_classifier.predict(lmtz_X_test)
print (classifier.best_params_)
print ("Accuracy Score : ",accuracy_score(lmtz_Y_test, lmtz_predicted))
print ("F1 Score : ",f1_score(lmtz_Y_test, lmtz_predicted))
print ("Confusion Matrix : \n",confusion_matrix(lmtz_Y_test, lmtz_predicted))

{'kernel': 'linear', 'gamma': 0.001, 'C': 1}
Accuracy Score :  0.6941176470588235
F1 Score :  0.7868852459016394
Confusion Matrix : 
 [[11 22]
 [ 4 48]]


### I ran above section of training several times and the difference reamains noticeable on all runs, so possibly linear kernel is best for both , but performance it much better without lemmatizing the data

### Here I am optimzing for F1 score in grid search as we have class imbalance , with more instances of apple as a company

## save, load and infer

In [42]:
# Save vocabulary
pickle.dump(vectorizer.vocabulary_,open("svmSem_Vocab.pkl","wb"))
# save svm model
pickle.dump(classifier,open("svmSem_Model.pkl","wb"))

In [43]:
transformer = TfidfTransformer()
loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("svmSem_Vocab.pkl", "rb")))

In [44]:
loaded_model = pickle.load(open("svmSem_Model.pkl", 'rb'))

In [45]:
comp = "apple launches new iphone"
fruit = 'understanding genes behind characteristics allow scientists perform knowledgeable selective breeding decoding genome also provided proof malus sieversii wild ancestor domestic applean issue longdebated scientific community'
testIDF = transformer.fit_transform(loaded_vec.fit_transform([comp, fruit]))
predicted = loaded_model.predict(testIDF)
print(predicted) # 1:company 0:fruit

[1 0]


# Use POS-tags as features

### Create train data with pos tags from raw data

### using part of speech tagging may help improving accuracy further for the current model, as we can have many sentences where apple as a company performs some action but very few sentences will have apple as fruit performing some action

In [17]:
import pandas as pd
import os
from nltk.corpus import stopwords
from IPython.display import display

from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,classification_report
import pickle

In [40]:
#sent = 'steve jobs founded apple'
sent = 'apple is eaten by steve jobs'
words = sent.split(' ')

In [41]:
tokens = nltk.word_tokenize(sent)
tagSent = nltk.pos_tag(tokens)
tagSentW = nltk.pos_tag(words)
(tokens,words)

(['apple', 'is', 'eaten', 'by', 'steve', 'jobs'],
 ['apple', 'is', 'eaten', 'by', 'steve', 'jobs'])

In [42]:
(tagSent,tagSentW)


([('apple', 'NN'),
  ('is', 'VBZ'),
  ('eaten', 'VBN'),
  ('by', 'IN'),
  ('steve', 'NN'),
  ('jobs', 'NNS')],
 [('apple', 'NN'),
  ('is', 'VBZ'),
  ('eaten', 'VBN'),
  ('by', 'IN'),
  ('steve', 'NN'),
  ('jobs', 'NNS')])

In [43]:
stopList = list(stopwords.words('english')) # Stopwords
tagged_sentence = [ word+'_'+tag for word,tag in tagSent if word not in stopList]
tagged_sentence

['apple_NN', 'eaten_VBN', 'steve_NN', 'jobs_NNS']

## Create pos-tagged data

In [63]:
stopList = list(stopwords.words('english')) # Stopwords
spclCharList  = ['#','^','!','(',')','`','~','%','/','\\' ,'{','}',']','[','.',',',':',';','"','-','°','@']

fruit_file = os.path.join(os.getcwd(),"TrainData\\apple-fruit.txt")
company_file = os.path.join(os.getcwd(),"TrainData\\apple-computers.txt")
trainDataText = open(os.path.join(os.getcwd(),"TrainData\\TrainPosData.txt"),'w')

with open(fruit_file,'r',encoding="utf-8") as input:
    paragraphs = input.read().split("\n\n")

In [64]:
datum =''
for para in paragraphs:
    for ch in spclCharList:
        para = para.replace(ch,'')
    para = ''.join([i for i in para if not i.isdigit()])
    para = para.lower().strip().split()
    para = list(filter(lambda x: x != '', para))
    para = nltk.pos_tag(para)
    para = [ word+'_'+tag for word,tag in para if word not in stopList]
    para = ' '.join(str(w) for w in para)
    datum += para +'\t'+'fruit'+'\n'

with open(company_file,'r',encoding="utf-8") as input:
    paragraphs = input.read().split("\n\n")

for para in paragraphs:
    for ch in spclCharList:
        para = para.replace(ch,'')
    para = ''.join([i for i in para if not i.isdigit()])
    para = para.lower().strip().split()
    para = list(filter(lambda x: x != '', para))
    para = nltk.pos_tag(para)
    para = [ word+'_'+tag for word,tag in para if word not in stopList]
    para = ' '.join(str(w) for w in para)
    datum += para +'\t'+'company'+'\n'

trainDataText.write(datum)
trainDataText.close()

In [65]:
paragraphs = []
label = []
trainDataText = open(os.path.join(os.getcwd(),"TrainData\\TrainPosData.txt"),'r')

In [66]:
dfPos = pd.DataFrame(columns=['paragraph', 'label'])
for line in trainDataText.readlines():
    line = line.rstrip('\n').split('\t')
    paragraphs.append(line[0])
    l = 1 if line[1] == 'company' else 0
    label.append(1 if line[1] == 'company' else 0)
    dfPos = dfPos.append({'paragraph':line[0],'label' :l},ignore_index=True)

In [67]:
#from nltk.stem.wordnet import WordNetLemmatizer
#lmtzr = WordNetLemmatizer()
#dfPosLmtz = pd.DataFrame(columns=['lmtzPara', 'label'])
#dfPosLmtz['lmtzPara'] = dfPos['paragraph'].apply(lambda x: " ".join([lmtzr.lemmatize(word) for word in x.split()]))
#dfPosLmtz['label'] = dfPos['label']

In [251]:
ngram = (1,3)
maxf= 1200

## Run below

In [289]:
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

In [290]:
vectorizer = TfidfVectorizer("english",ngram_range = ngram, max_features = maxf)
X = vectorizer.fit_transform(paragraphs)
#type(vectorizer.get_feature_names()[:11])
#type(vectorizer.vocabulary_)
#vectorizer.get_feature_names()[:11]
#vectorizer.idf_

In [291]:
X_train, X_test, Y_train, Y_test = train_test_split(X, label, stratify=label, test_size=.4)

In [292]:
hyper_parameters = [{'kernel': ['linear'], 'gamma': [1e-3, 1e-4],'C': [1, 10, 100, 1000]}]

### Train

In [293]:
classifier = GridSearchCV(SVC(), hyper_parameters, cv=5, scoring='f1')
classifier.fit(X_train, Y_train)
best_param = classifier.best_params_
best_classifier = classifier.best_estimator_
print('best param : ',best_param)

best param :  {'kernel': 'linear', 'gamma': 0.001, 'C': 1}


g:\anaconda3\envs\python35\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


### Predict

In [294]:
predicted = classifier.predict(X_test)
print (classifier.best_params_)
print ("Accuracy Score : ",accuracy_score(Y_test, predicted))
print ("F1 Score : ",f1_score(Y_test, predicted))
print ("Confusion Matrix : \n",confusion_matrix(Y_test, predicted))

{'kernel': 'linear', 'gamma': 0.001, 'C': 1}
Accuracy Score :  0.9647058823529412
F1 Score :  0.9714285714285713
Confusion Matrix : 
 [[31  2]
 [ 1 51]]


In [295]:
print ('\nClasification report:\n', classification_report(Y_test, predicted))


Clasification report:
               precision    recall  f1-score   support

           0       0.97      0.94      0.95        33
           1       0.96      0.98      0.97        52

   micro avg       0.96      0.96      0.96        85
   macro avg       0.97      0.96      0.96        85
weighted avg       0.96      0.96      0.96        85

